In [134]:
import pandas
import numpy as np
import re
import math
import xlrd

In [212]:
test_csv = pandas.read_csv('Folder 3 - 1_18.csv')
test_xls = pandas.ExcelFile('Folder 3.xlsx')

position = '3'

In [ ]:
def convertDate(date):
    date = '2019-' + date[0:-2] + '-' + date[-2:]
    return date
#date = '2019-' + test_date[-4:].replace('/', '-')

In [98]:
# Turn a hh:mm:ss string to number of seconds
# hh: piece is optional

HHMMSS_PAT = re.compile("(\d+)??:??(\d+):(\d+)$")
def hhmmss_to_seconds(s):
    m = HHMMSS_PAT.match(s)
    g = m.groups()
    s = 0
    n = len(g)
    for i in range(n):
        if g[n-i-1] is not None:
            s += 60**i * int(g[n-i-1])
    return s

In [163]:
def convertTime(time):
    if time == time:
        converted = []
        time = time.replace(" ", "")
        for item in time.split(','):
            time_to_convert = item.split('/')[0]
            converted.append(hhmmss_to_seconds(time_to_convert))
    else:
        converted = []
    return converted

In [213]:
def parseSheet(test_csv, date, position):
    test_dict = {}

    for item in test_csv.iterrows():
        if item[0]>1:
            seizures = {}
            date_dict = {}
            pos_dict = {}
            seizures[mouse_labels[0]] = convertTime(item[1][2])
            seizures[mouse_labels[1]] = convertTime(item[1][3])
            seizures[mouse_labels[2]] = convertTime(item[1][4])
            date_dict[date] = seizures
            pos_dict[position] = date_dict
            test_dict[item[1][0]] = pos_dict
    
    return test_dict

In [217]:
all_sheets = {}

for sheet in test_xls.sheet_names:
    this_sheet = pandas.read_excel('Folder 3.xlsx', sheet_name=sheet)
    date = convertDate(sheet)
    position = '3'
    new_dict = parseSheet(this_sheet, date, position)
    all_sheets.update(new_dict)

In [218]:
import json
with open('seizure_annotations_3.json', 'w') as outfile:
    json.dump(all_sheets, outfile)